# 02 — API Integration (External Data → Reasoning)

외부 API (예: OpenWeatherMap)를 호출해 **실시간 데이터**를 받아 모델이 **컨텍스트로 활용**하도록 합니다.

## 준비
- `OPENWEATHER_API_KEY` 환경 변수를 설정하세요.
- 도시명을 입력하면 현재 날씨를 조회 후 한 줄 요약을 생성합니다.


In [ ]:
from dotenv import load_dotenv
load_dotenv()

import os, requests
from openai import OpenAI

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENWEATHER_API_KEY = os.getenv("OPENWEATHER_API_KEY")
assert OPENAI_API_KEY, "OPENAI_API_KEY가 필요합니다."
assert OPENWEATHER_API_KEY, "OPENWEATHER_API_KEY가 필요합니다."

client = OpenAI(api_key=OPENAI_API_KEY)

In [ ]:
def get_weather(city: str) -> dict:
    url = "https://api.openweathermap.org/data/2.5/weather"
    params = { "q": city, "appid": os.getenv("OPENWEATHER_API_KEY"), "units": "metric", "lang": "kr" }
    r = requests.get(url, params=params, timeout=20)
    r.raise_for_status()
    return r.json()

In [ ]:
def summarize_weather(city: str, weather_json: dict, model="gpt-4o-mini") -> str:
    desc = weather_json['weather'][0]['description']
    temp = weather_json['main']['temp']
    msg = f"{city} 현재 날씨는 '{desc}', 기온은 {temp}°C 입니다. 이를 참고해 1) 짧은 일기 한 문장 2) 추천 활동 1가지를 제안하세요."
    completion = client.chat.completions.create(
        model=model,
        messages=[{"role":"user","content": msg}],
        max_tokens=200,
        temperature=0.7
    )
    return completion.choices[0].message.content.strip()

## Try it

In [ ]:
city = "Seoul"
w = get_weather(city)
summarize_weather(city, w)